In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

# 벡터 DB : Chroma vs Pinecone
- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB
    (Pinecone console에 api key 생성 -> .env (PINECONE_API_KEY등록)

# 0. 패키지 설치

In [ ]:
%pip install -q pinecone-client langchain-pinecone

# 1. Knowledge Base 구성을 위한 데이터 생성

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/with_markdown.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
len(document_list)

In [ ]:
# embedding : upstage embedding-query
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
    # model="embedding-query"
)

In [ ]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = "tax-index-table"
database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name
)
# 업로드한 벡터DB 가져올 때
# database = PineconeVectorStore(
#     embedding=embedding, # 질문을 임베딩하여 유사도 검색
#     index_name=index_name
# )

# 2. 제공되는 prompt를 활용하여 답변 생성

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke({'query':query})
ai_message